In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import nltk
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/Hollows.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['chapter', 'dark', 'lord', 'ascending', 'appeared', 'nowhere', 'yards', 'apart', 'narrow', 'moonlit']
size corpus: 115295
size of unique words: 11342


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('harry', 3007), ('that', 2062), ('said', 1977), ('they', 1199)]
Sample data [456, 81, 131, 9074, 311, 1444, 1583, 517, 1323, 6614] ['chapter', 'dark', 'lord', 'ascending', 'appeared', 'nowhere', 'yards', 'apart', 'narrow', 'moonlit']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  117.660697937
Nearest to place: increasing, became, punctuated, painting, stab, built, grindelwald, y-yourself,
Nearest to about: emotion, atmospheric, dogs, tassel, brushing, glade, birds, interest,
Nearest to last: members, bamboozled, resented, keeps, angrier, kills, tidy, thicknesse,
Nearest to would: realms, sparkling, londoners, involves, moonlit, flamel, upside, filing,
Nearest to behind: purpose, avoiding, boulders, sleekly, weekend, indeed, surface, fence,
Nearest to dark: glimmering, fondness, commotion, lashes, harried, sons, knees, decoys,
Nearest to only: sped, hill, whining, aimlessly, silky, uncorked, cellars, supposing,
Nearest to through: squit, dragging, weeks, hillside, salazar, seven, boast, coil,
Nearest to know: floor-length, warmth, communicating, tonks, daredevil, defaming, er-phineas, poked,
Nearest to face: anger, enthusiastically, sources, distantly, loomed, cheerfully, cigarette, grab,
Nearest to told: triumphs, detonat

Average loss at step  50000 :  3.09607201588
Nearest to place: increasing, declaims, misshapen, sood, oversleep, tidier, built, punctuated,
Nearest to about: weepy, scraps, glade, sisters, true, rights, endo, fresher,
Nearest to last: members, april, handling, twenty-four, comes, gellert, settled, whenever,
Nearest to would: wouldnt, might, realms, seemed, dispensable, summoned, affection, anythin,
Nearest to behind: purpose, streets, vapor, concentrating, step, proviso, biding, draperies,
Nearest to dark: glimmering, lunged, harried, gates, authors, warmly, respectfully, flung,
Nearest to only: sped, vain, protective, contradict, invisible, legendary, glisseo, muffled,
Nearest to through: hillside, superfast, coil, idiots, clattering, parchment, squawking, excursions,
Nearest to know: sleekly, revelry, doubt, opportunity, proved, harsh, remember, exchanging,
Nearest to face: heavily, unnaturally, pummeled, invented, mishap, immaterial, anger, awake,
Nearest to told: voted, threw, depl

Average loss at step  92000 :  2.70082075548
Average loss at step  94000 :  2.70565849078
Average loss at step  96000 :  2.70767602068
Average loss at step  98000 :  2.68511972517
Average loss at step  100000 :  2.69541834688
Nearest to place: oversleep, straightaway, sood, gamboling, declaims, amidst, reliance, whimpering,
Nearest to about: glade, -yes, endo, fresher, weepy, rights, arrested, declare,
Nearest to last: handling, members, april, chivalrous, twenty-four, loudets, secluded, warm,
Nearest to would: might, realms, wouldnt, dispensable, drag, amplified, sting, seem,
Nearest to behind: purpose, biding, proviso, unproctected, pomfrey, draperies, streets, dogbreath,
Nearest to dark: glimmering, entrances, semidarkness, respectfully, warmly, authors, poles, displeases,
Nearest to only: invisible, sped, contradict, legendary, glisseo, matching, vain, things,
Nearest to through: coil, hillside, dawn, squawking, week, reviewed, idiots, superfast,
Nearest to know: sleekly, revelry, 

Average loss at step  142000 :  2.47908704239
Average loss at step  144000 :  2.52846115983
Average loss at step  146000 :  2.46673949254
Average loss at step  148000 :  2.48298943102
Average loss at step  150000 :  2.48517858928
Nearest to place: oversleep, amidst, sood, straightaway, declaims, reliance, currently, gamboling,
Nearest to about: -yes, glade, rights, endo, fresher, grindelvalds, dots, tested,
Nearest to last: april, handling, blamed, chivalrous, members, ginny, whenever, uncertainly,
Nearest to would: malediction, realms, wouldnt, might, amplified, nonsense, drag, anythin,
Nearest to behind: biding, unproctected, draperies, dogbreath, bludgering, avoiding, possesses, proviso,
Nearest to dark: respectfully, entrances, glimmering, poles, quantity, gargoyles, gates, harried,
Nearest to only: invisible, legendary, contradict, glisseo, sped, fright, things, unzipping,
Nearest to through: coil, squawking, idiots, hillside, week, reviewed, doors, grime,
Nearest to know: revelry

Average loss at step  192000 :  2.38567991585
Average loss at step  194000 :  2.40905885255
Average loss at step  196000 :  2.35519608951
Average loss at step  198000 :  2.42101136822


In [8]:
with open('hollow-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('hollow-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)